In [1]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import re
import datetime as dt
import pickle

from tqdm.notebook import tqdm_notebook

# Writing multiple dataframes to worksheets using Pandas and XlsxWriter
import xlsxwriter
import glob

import re
from collections import Counter
# from textblob import TextBlob
import langid
import iso639

In [2]:
data = pd.read_pickle("url_KZ_RU_UA.pkl") 

In [3]:
data.shape

(92960, 4)

In [4]:
data.head()

,URL,Сontent,Keywords,Name
0,http://youtube.com/channel/UCIPi6u4H3NZFuffKAl...,Assalomu alaykumKanalimizga xush kelibsizАЗИЗ ...,"трудовые мигранты"" ""аср медиа"" ""asr media"" ""по...",ASR MEDIA
1,http://youtube.com/channel/UCjdHWLIKJNHsqaNJAF...,Официальный сайт : https://sho...,"qiziqarli videolar"" haqida ""shov-shuv uz",Shov- Shuv UZ
2,http://youtube.com/channel/UCSU6G0iZ5MI7bIM8pT...,Реклама на канале: pr@rumonkey.ruСмотрите шоу ...,"дорого богато"" обзор ""самый дорогой"" приветтач...",ПриветТачка
3,http://youtube.com/channel/UC2ADOzFv97QSk5NTN_...,NaN,сериалы фильмы кино новинки комедии мелодрамы ...,КиноПробка
4,http://youtube.com/channel/UCrGdxtTRGxAdMtz9xL...,«Прокурорская проверка» — это детективная реал...,NaN,Прокурорская Проверка


In [5]:
# Removing Empty rows
data.dropna(subset=['Сontent', 'Keywords', 'Name'], how='all', inplace=True)

In [6]:
data.drop_duplicates(subset=['URL'], inplace=True)

In [7]:
# Combination
data['text'] = data['Сontent'].astype(
    str) + ' ' + data['Keywords'].astype(str) + ' ' + data['Name'].astype(str)

In [8]:
data.head()

,URL,Сontent,Keywords,Name,text
0,http://youtube.com/channel/UCIPi6u4H3NZFuffKAl...,Assalomu alaykumKanalimizga xush kelibsizАЗИЗ ...,"трудовые мигранты"" ""аср медиа"" ""asr media"" ""по...",ASR MEDIA,Assalomu alaykumKanalimizga xush kelibsizАЗИЗ ...
1,http://youtube.com/channel/UCjdHWLIKJNHsqaNJAF...,Официальный сайт : https://sho...,"qiziqarli videolar"" haqida ""shov-shuv uz",Shov- Shuv UZ,Официальный сайт : https://sho...
2,http://youtube.com/channel/UCSU6G0iZ5MI7bIM8pT...,Реклама на канале: pr@rumonkey.ruСмотрите шоу ...,"дорого богато"" обзор ""самый дорогой"" приветтач...",ПриветТачка,Реклама на канале: pr@rumonkey.ruСмотрите шоу ...
3,http://youtube.com/channel/UC2ADOzFv97QSk5NTN_...,NaN,сериалы фильмы кино новинки комедии мелодрамы ...,КиноПробка,nan сериалы фильмы кино новинки комедии мелодр...
4,http://youtube.com/channel/UCrGdxtTRGxAdMtz9xL...,«Прокурорская проверка» — это детективная реал...,NaN,Прокурорская Проверка,«Прокурорская проверка» — это детективная реал...


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76530 entries, 0 to 92959
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   URL       76530 non-null  object
 1   Сontent   58706 non-null  object
 2   Keywords  45415 non-null  object
 3   Name      76527 non-null  object
 4   text      76530 non-null  object
dtypes: object(5)
memory usage: 3.5+ MB


## Channel language definitions

In [10]:
df_lang = data.copy()
df_lang['keys'] = df_lang['Сontent'].astype(
    str) + ' ' + df_lang['Keywords'].astype(str)
df_lang = df_lang[['URL', 'keys']]
# Delete Nan
df_lang['keys'] = df_lang['keys'].apply(lambda x: x.replace('nan', ''))
#  Removing Empty rows
df_lang = df_lang[df_lang['keys'].map(len) > 1]
df_lang.shape


def lang(el):
    b = langid.classify(el)[0]
    return b

In [11]:
tqdm_notebook.pandas()
df_lang['language_code'] = df_lang['keys'].progress_map(lang)

  0%|          | 0/62893 [00:00<?, ?it/s]

In [12]:
df_nan_keys = data[data['Сontent'].isna()][['URL', 'Name']].copy()
#  Removing Empty rows
df_nan_keys = df_nan_keys[df_nan_keys['Name'].map(len) > 1]

In [13]:
df_nan_keys['language_code'] = df_nan_keys['Name'].progress_map(lang)

  0%|          | 0/17820 [00:00<?, ?it/s]

In [14]:
df_lang_final = pd.concat([df_lang[['URL', 'language_code']], df_nan_keys[['URL', 'language_code']]])

In [15]:
df_lang_final.shape

(80713, 2)

In [16]:
data.drop_duplicates(subset=['URL'], keep='first',inplace=True)

In [17]:
df_lang_final.shape

(80713, 2)

In [18]:
from pycountry import languages

df_lang_final['language'] = df_lang_final['language_code'].apply(
    lambda x: languages.get(alpha_2=x).name)

In [19]:
df_lang_final.head()

,URL,language_code,language
0,http://youtube.com/channel/UCIPi6u4H3NZFuffKAl...,ru,Russian
1,http://youtube.com/channel/UCjdHWLIKJNHsqaNJAF...,ru,Russian
2,http://youtube.com/channel/UCSU6G0iZ5MI7bIM8pT...,ru,Russian
3,http://youtube.com/channel/UC2ADOzFv97QSk5NTN_...,ru,Russian
4,http://youtube.com/channel/UCrGdxtTRGxAdMtz9xL...,ru,Russian


In [20]:
data = pd.merge(data, df_lang_final, how='left', on=['URL'])

In [21]:
# Removing characters
p = re.compile(r'[^\w\s]+')
data['text'] = [p.sub('', x) for x in data['text'].tolist()]

In [22]:
# Lower str
data['text'] = data['text'].str.lower()

## First of all

In [23]:
data.loc[data['text'].str.contains(
    r'\bmakes|motivation|motivationalترفيهية|hobby|mystery|развлекательное|vibes|vibe|hội|bmx|paul\b'), 'CAT'] = 'Развлечения'

data.loc[data['text'].str.contains(
    r'\bchallenge|vines|tiktok compilations|challenge|vine|tiktoks|interesting|facts\b'), 'CAT'] = 'Развлечения'

data.loc[data['text'].str.contains(
    r'\bпроверки лайфхаков|скетчи|скетч|anime|kidsthis|dolls|софия|طف|çocuk|bebek|طفل\b'), 'CAT'] = 'Детское'

data.loc[data['text'].str.contains(
    r'\bлайфхак|бред|литвин|动漫\b'), 'CAT'] = 'Детское'

data.loc[data['text'].str.contains(r'\bлайфхак\B'), 'CAT'] = 'Детское'

data.loc[data['text'].str.contains(
    r'\bчеллендж|челленджы|челленджи|a4|south park|bebé|niña|niño|амелька\b'), 'CAT'] = 'Детское'
data.loc[data['text'].str.contains(
    r'\bэксперимент|baby|truyện\b'), 'CAT'] = 'Детское'

data.loc[data['text'].str.contains(r'\bэксперимент\B'), 'CAT'] = 'Детское'

data.loc[data['text'].str.contains(
    r'\bмаша|diana|даша|диана|slime|поделки|фрост|심슨|simpson\b'), 'CAT'] = 'Детское'

data.loc[data['text'].str.contains(
    r'\bполёты|работаю|спецтехника|automotive|bikes|war|戰爭|machine|knight|trucks|discoveries\b'), 'CAT'] = 'Мужские Хобби'

data.loc[data['text'].str.contains(
    r'\bcinema|horror|cinéma|season|seasons|понять простить|hd|شوف|got talent\b'), 'CAT'] = 'Кино и Сериалы'

data.loc[data['text'].str.contains(
    r'\btv|24|тв|федеральный|ведущий|катаклизмы|русская мощь|quốc|episode|episodes|برنامه\b'), 'CAT'] = 'Новости и политика'

data.loc[data['text'].str.contains(
    r'\Btv|yangiliklar|public affairs\b'), 'CAT'] = 'Новости и политика'

data.loc[data['text'].str.contains(
    r'\bоппозиц|пропаган|кремл\B'), 'CAT'] = 'Новости и политика'

data.loc[data['text'].str.contains(
    r'\bспівачка|pop|lyric|hop|dance|гурт|рокгурт|mc|soundtrack|band|elvis|tracks|punk|hit|співак|playlists\b')
         , 'CAT'] = 'Музыка'

data.loc[data['text'].str.contains(
    r'\bkaraoke|track|kanye west|الأغاني|spotify|melodic|rádió|playlist|музика|الفنان|rammstein|machine gun kelly\b')
         , 'CAT'] = 'Музыка'

data.loc[data['text'].str.contains(
    r'\bcoldplay|nirvana|музикант|slipknot|концерт|album|라디오\b'), 'CAT'] = 'Музыка'

data.loc[data['text'].str.contains(r'\Bхит\b'), 'CAT'] = 'Музыка'

data.loc[data['text'].str.contains(
    r'\bхит|артист|концерт\B'), 'CAT'] = 'Музыка'

data.loc[data['text'].str.contains(r'\Bхит\B'), 'CAT'] = 'Музыка'

data.loc[data['text'].str.contains(
    r'\bуход|кожа|кожей|волосы|волосами|наталья|girl|татьяна|елена|светлана|elena|марина\b'), 'CAT'] = 'Женские хобби'

data.loc[data['text'].str.contains(
    r'\bоксана|юлия|певица|александра|maria|раиса|галина|nastya|анастасия|cake|стихи|anastasia|виктория|\
    натали|ольга|рада|mama\b'), 'CAT'] = 'Женские хобби'

data.loc[data['text'].str.contains(r'\bанна|tanya\b'), 'CAT'] = 'Женские хобби'

data.loc[data['text'].str.contains(r'\bdanc\B'), 'CAT'] = 'Музыка'

data.loc[data['text'].str.contains(
    r'\bdj|bass|исполнитель|muzik|dance|dancing|fm\b'), 'CAT'] = 'Музыка'

## People and blogs

In [24]:
data.loc[data['text'].str.contains(
    r'\bговорим|говорю|аудиорассказы|жизнь|судьба|меня зовут|vlog|дпс|tutorial|art|dance|streams|일상|yêu|\
    love|يوميات|đời|이야기를|해외반응|ảnh|عشق|الشيخ\b'), 'CAT'] = 'Люди и блоги'

data.loc[data['text'].str.contains(
    r'\bуслышать|увидеть|трансляции|стримы|влог|блог|стрим|мамы|мама|любовь|drive|travel|technology|tình|\
    youtuber|ثقافية|세상을|اسرار|여행\b'), 'CAT'] = 'Люди и блоги'

data.loc[data['text'].str.contains(
    r'\bиз жизни|любви|чатрулетка|видеочат|candy|magic|amor|рецепты|лайф|boling|history|lovely|семья|리니지\
    m|mission\b'), 'CAT'] = 'Люди и блоги'

data.loc[data['text'].str.contains(
    r'\bдневник|семейный|traveling|домашний|life|family|story|podcast|العالم|stories|journey|người|twitch|قصص|\
    sống|building\b'), 'CAT'] = 'Люди и блоги'

data.loc[data['text'].str.contains(
    r'\Bрассказы|قصص|истории|киссахо|киссахои|кисахои|историй|культура|говорят|интервью|мнения|аналитика|\
    мнение|ютуберы|истории\b'), 'CAT'] = 'Люди и блоги'

data.loc[data['text'].str.contains(
    r'\bрасска|отношени|знакомст|социальн|биограф|biograph|жизненны\B'), 'CAT'] = 'Люди и блоги'

data.loc[data['text'].str.contains(
    r'\bcouple|guide|lover|speak|historias|vlogger|شديدة|useful tips|tips\b'), 'CAT'] = 'Люди и блоги'

## Entertainment

In [25]:
data.loc[data['text'].str.contains(
    r'\bинтересные факты|qiziqarli|фактах|факту|факти|факты|удивительные факты|интересные истории|\
    фактов|ترفيهي\b'), 'CAT'] = 'Развлечения'

data.loc[data['text'].str.contains(
    r'\bлафхак|секреты|пранк|рекция|реакции|prank|интересные|переезд|о жизни|фото|ghost|scary|creepy\b')
         , 'CAT'] = 'Развлечения'

data.loc[data['text'].str.contains(
    r'\bреакция|секрет|советы|мистич|рассказыва|переех\B'), 'CAT'] = 'Развлечения'

data.loc[data['text'].str.contains(
    r'\bтоп5|топ 5|топ10|топ 10|топы|топ|илон маск|reaction\b'), 'CAT'] = 'Развлечения'

data.loc[data['text'].str.contains(
    r'\bмир|мире|планета|планеты|космос|заброшенный|покинутый|приключения|тайны\b'), 'CAT'] = 'Развлечения'

data.loc[data['text'].str.contains(
    r'\bобзор|обзоры|обзор техники|meme|gadgets|legend\b'), 'CAT'] = 'Развлечения'

data.loc[data['text'].str.contains(
    r'\bсмартфон|телефон|гаджет|гаджеты|гаджетов|android|хобби|blog|trend|отдых|entertainment|руками|craft|making\b')
         , 'CAT'] = 'Развлечения'

data.loc[data['text'].str.contains(
    r'\bсмартфон|телефон|аксессуар|фотогра|развлече\B'), 'CAT'] = 'Развлечения'

data.loc[data['text'].str.contains(
    r'\bпутешеств|приключен\B'), 'CAT'] = 'Развлечения'

data.loc[data['text'].str.contains(
    r'\bсоциальные эксперименты|социальный эксперимент|эксперементы|идеи|homemade|crash\b'), 'CAT'] = 'Развлечения'

data.loc[data['text'].str.contains(
    r'\bаудиокниги|аудиокнига|гидравлический|top|live|страшные|secrets|planet|hand|adventures\b'), 'CAT'] = 'Развлечения'

data.loc[data['text'].str.contains(
    r'\bмистика|тамада|entertain|analysis\b'), 'CAT'] = 'Развлечения'

data.loc[data['text'].str.contains(r'\Bмистика\b'), 'CAT'] = 'Развлечения'

data.loc[data['text'].str.contains(r'\bмистик\B'), 'CAT'] = 'Развлечения'

## Education

In [26]:
data.loc[data['text'].str.contains(
    r'\beducation|уроки|darslar|дарсхо|dersler|дәрістер|обучение|обучения|تعلم\b'), 'CAT'] = 'Образования'
data.loc[data['text'].str.contains(
    r'\bизучения|масстер клас|видео урок|видео уроки|пдд|саморазвитие\b'), 'CAT'] = 'Образования'
data.loc[data['text'].str.contains(
    r'\bобуча|изучени|масстеркла|видеоурок\B'), 'CAT'] = 'Образования'


data.loc[data['text'].str.contains(
    r'\bизучать английский|английский с нуля|английский язык|học|learning|آموزش\b'), 'CAT'] = 'Образования'
data.loc[data['text'].str.contains(
    r'\bвыучить английский|language learning|иностранный язык\b'), 'CAT'] = 'Образования'
data.loc[data['text'].str.contains(
    r'\bпреподаватель английского|разговорный английский|уроки английского\b'), 'CAT'] = 'Образования'
data.loc[data['text'].str.contains(
    r'\bанглийскому языку|носитель языка|выучить|изучения|teacher\b'), 'CAT'] = 'Образования'

data.loc[data['text'].str.contains(
    r'\bмедитация|medical|meditation|clinic\b'), 'CAT'] = 'Здоровье'
data.loc[data['text'].str.contains(r'\Bмедитация\b'), 'CAT'] = 'Здоровье'
data.loc[data['text'].str.contains(r'\bмедит\B'), 'CAT'] = 'Здоровье'

data.loc[data['text'].str.contains(
    r'\bйога|د|healing|الدكتور|health|yoga\b'), 'CAT'] = 'Здоровье'
data.loc[data['text'].str.contains(r'\Bйога\b'), 'CAT'] = 'Здоровье'
data.loc[data['text'].str.contains(r'\bйога\B'), 'CAT'] = 'Здоровье'

data.loc[data['text'].str.contains(
    r'\bврача|клиника|лечение|врач|психолог|диета|диеты|диет|массаж\b'), 'CAT'] = 'Здоровье'
data.loc[data['text'].str.contains(
    r'\Bмедицины|медицина|лечение|питание|профилактика\b'), 'CAT'] = 'Здоровье'
data.loc[data['text'].str.contains(
    r'\bмедицин|клиничес|психолог|психиче|заболеван|неврол|профилакти|диетич|массаж\B'), 'CAT'] = 'Здоровье'

## Finance

In [27]:
data.loc[data['text'].str.contains(r'\bфинансы|финансовая|trading|гроші|экономить|crypto\b'), 'CAT'] = 'Финансы'
data.loc[data['text'].str.contains(r'\Bфинансы|crypto\b'), 'CAT'] = 'Финансы'
data.loc[data['text'].str.contains(r'\bфинан|crypto\B'), 'CAT'] = 'Финансы'
data.loc[data['text'].str.contains(r'\Bфинан|crypto\B'), 'CAT'] = 'Финансы'


data.loc[data['text'].str.contains(r'\bmoney\b'), 'CAT'] = 'Финансы'
data.loc[data['text'].str.contains(r'\bbusiness\b'), 'CAT'] = 'Финансы'
data.loc[data['text'].str.contains(r'\Bmoney\b'), 'CAT'] = 'Финансы'
data.loc[data['text'].str.contains(r'\bmoney\B'), 'CAT'] = 'Финансы'
data.loc[data['text'].str.contains(r'\BmoneyB'), 'CAT'] = 'Финансы'


data.loc[data['text'].str.contains(r'\bmoney|finance\b'), 'CAT'] = 'Финансы'
data.loc[data['text'].str.contains(r'\Bmoney\b'), 'CAT'] = 'Финансы'
data.loc[data['text'].str.contains(r'\bmoney\B'), 'CAT'] = 'Финансы'
data.loc[data['text'].str.contains(r'\BmoneyB'), 'CAT'] = 'Финансы'

data.loc[data['text'].str.contains(r'\bденьги\b'), 'CAT'] = 'Финансы'
data.loc[data['text'].str.contains(r'\Bденьги\b'), 'CAT'] = 'Финансы'
data.loc[data['text'].str.contains(r'\bденьг|busine\B'), 'CAT'] = 'Финансы'
data.loc[data['text'].str.contains(r'\BденьгB'), 'CAT'] = 'Финансы'

data.loc[data['text'].str.contains(r'\bзаработать|заработок\b'), 'CAT'] = 'Финансы'
data.loc[data['text'].str.contains(r'\Bзаработать\b'), 'CAT'] = 'Финансы'
data.loc[data['text'].str.contains(r'\bзараб\B'), 'CAT'] = 'Финансы'
data.loc[data['text'].str.contains(r'\BзаработB'), 'CAT'] = 'Финансы'

data.loc[data['text'].str.contains(r'\bбизнес\b'), 'CAT'] = 'Финансы'
data.loc[data['text'].str.contains(r'\Bбизнес\b'), 'CAT'] = 'Финансы'
data.loc[data['text'].str.contains(r'\bбизне\B'), 'CAT'] = 'Финансы'
data.loc[data['text'].str.contains(r'\BбизнесB'), 'CAT'] = 'Финансы'

data.loc[data['text'].str.contains(r'\bбиткоин\b'), 'CAT'] = 'Финансы'
data.loc[data['text'].str.contains(r'\Bбиткоин\b'), 'CAT'] = 'Финансы'
data.loc[data['text'].str.contains(r'\bбиткоин\B'), 'CAT'] = 'Финансы'
data.loc[data['text'].str.contains(r'\BбиткоинB'), 'CAT'] = 'Финансы'

data.loc[data['text'].str.contains(r'\bкрипта|фондового рынка|фондовый рынок|трейдинг|схемы\b'), 'CAT'] = 'Финансы'
data.loc[data['text'].str.contains(r'\Bкрипта\b'), 'CAT'] = 'Финансы'
data.loc[data['text'].str.contains(r'\bкрипт\B'), 'CAT'] = 'Финансы'
data.loc[data['text'].str.contains(r'\bфондов\B'), 'CAT'] = 'Финансы'

## Humor

In [53]:
data.loc[data['text'].str.contains(
    r'\balan chikin chow|monilina|tsuriki show|jixcx|manchurek triplets|the mccartys|savage vlogs|\
    topper guild|daniel labelle|fk comedy tv|adam w|the one shu|mingweirocks|zhong|onevilage|\
    family the honest comedy|bugworkout sport in russia|el tío néstor|ray william johnson|\
    jkk entertainment|susy mouriz|cs bisht vines|the formal edit|i am mobo|khandeshi movies|\
    luis méndez|howieazy|ash magic|iker unzu|abhishekd91|shane nagy|vector|nachter|hassan khadair|\
    etv jabardasth|ندي و احمد|jeenieweenie|clepton|enaldinho|anime uz|vujae|golden unique boy|\
    pablo bruschi|familia parafuso solto|tiktoriki|johnmarc van wyk|julia fan|triggered insaan|\
    shemaroo comedy|çok güzel hareketler|東海オンエア|iamdaud14|javi luna|warintil official|\
    akash sagar|the tonight show starring jimmy fa|mohak meet|failarmy|abyan calief|chimkandi|\
    paje team|the mridul|juliano coração|wilson sinai|yerrikillerxd|fattv|desi version|кmedia|\
    telugu letest all|fabinvideos|affection|güldür güldür|mr bean|sergio jurado|teluguone|\
    chhotu ki masti|smile squad comedy|luisito comunica|morgan wallen|arif muhammad|\
    enchufetv|buttered side down|omgitsnikefinesse|juan de montreal  asia fan|mis pequeños actores|\
    franco escamilla|benoftheweek|jakesterguy|singto numchok official|deny montana|film star celebrity|\
    charlie berens|clab33|dougmar|shiloh  bros|ashish mishra|fap tv|thesaudireporters  سعودي ر|\
    pablo toneti|focus et  فوكست|dazn japan\b'), 'CAT'] = 'Юмор'

In [54]:
data.loc[data['text'].str.contains(r'\bюмор|humor|komik|comedian|comedians|搞笑\B'), 'CAT'] = 'Юмор'
data.loc[data['text'].str.contains(r'\bюмор|재밌는|搞笑|funniest\b'), 'CAT'] = 'Юмор'
data.loc[data['text'].str.contains(r'\bюмор\B'), 'CAT'] = 'Юмор'
data.loc[data['text'].str.contains(r'\Bюмор\b'), 'CAT'] = 'Юмор'

data.loc[data['text'].str.contains(r'\bприкол|joke\B'), 'CAT'] = 'Юмор'
data.loc[data['text'].str.contains(r'\bприколы|jokes|재미있는\b'), 'CAT'] = 'Юмор'
data.loc[data['text'].str.contains(r'\bприколы\b'), 'CAT'] = 'Юмор'

data.loc[data['text'].str.contains(r'\bанекдоты\b'), 'CAT'] = 'Юмор'
data.loc[data['text'].str.contains(r'\bанекдот\b'), 'CAT'] = 'Юмор'
data.loc[data['text'].str.contains(r'\bанекд\B'), 'CAT'] = 'Юмор'
data.loc[data['text'].str.contains(r'\bпранки|пранк|prank|pranks\b'), 'CAT'] = 'Юмор'
data.loc[data['text'].str.contains(r'\bсмешное|смешно|поржать|ржать|смешные|смешное|ржачные|квн\b'), 'CAT'] = 'Юмор'

## Games

In [55]:
data.loc[data['text'].str.contains(r'\bgame|اللعب|لعبه|trò chơi|게임\b'), 'CAT'] = 'Игры'
data.loc[data['text'].str.contains(r'\bgames\b'), 'CAT'] = 'Игры'
data.loc[data['text'].str.contains(r'\Bgame|trò chơi\b'), 'CAT'] = 'Игры'
data.loc[data['text'].str.contains(r'\Bgames\b'), 'CAT'] = 'Игры'
data.loc[data['text'].str.contains(r'\bgame|trò chơi\B'), 'CAT'] = 'Игры'
data.loc[data['text'].str.contains(r'\bgamer\B'), 'CAT'] = 'Игры'
data.loc[data['text'].str.contains(r'\bgames\B'), 'CAT'] = 'Игры'
data.loc[data['text'].str.contains(r'\BgameB'), 'CAT'] = 'Игры'

data.loc[data['text'].str.contains(r'\bgaming\b'), 'CAT'] = 'Игры'
data.loc[data['text'].str.contains(r'\Bgaming\b'), 'CAT'] = 'Игры'
data.loc[data['text'].str.contains(r'\bgaming\B'), 'CAT'] = 'Игры'
data.loc[data['text'].str.contains(r'\BgamingB'), 'CAT'] = 'Игры'


data.loc[data['text'].str.contains(r'\bgameplay|spielweise\b'), 'CAT'] = 'Игры'
data.loc[data['text'].str.contains(r'\Bgameplay\b'), 'CAT'] = 'Игры'
data.loc[data['text'].str.contains(r'\bgameplay\B'), 'CAT'] = 'Игры'
data.loc[data['text'].str.contains(r'\Bgameplay\B'), 'CAT'] = 'Игры'

data.loc[data['text'].str.contains(r'\bигры\b'), 'CAT'] = 'Игры'
data.loc[data['text'].str.contains(r'\bигре\b'), 'CAT'] = 'Игры'
data.loc[data['text'].str.contains(r'\bигр\b'), 'CAT'] = 'Игры'
data.loc[data['text'].str.contains(r'\Bигры\b'), 'CAT'] = 'Игры'
data.loc[data['text'].str.contains(r'\bигр\B'), 'CAT'] = 'Игры'


data.loc[data['text'].str.contains(r'\bminecraft\b'), 'CAT'] = 'Игры'
data.loc[data['text'].str.contains(r'\Bminecraft\b'), 'CAT'] = 'Игры'
data.loc[data['text'].str.contains(r'\bminecraft\B'), 'CAT'] = 'Игры'
data.loc[data['text'].str.contains(r'\Bminecraft\B'), 'CAT'] = 'Игры'

data.loc[data['text'].str.contains(r'\bгеймер|rust\b'), 'CAT'] = 'Игры'
data.loc[data['text'].str.contains(r'\bгейм\B'), 'CAT'] = 'Игры'

data.loc[data['text'].str.contains(r'\bgta|bstandoff|csgo|dota 2|дота 2|wot|dota|pubg|warface|прохождение\b'), 'CAT'] = 'Игры'
data.loc[data['text'].str.contains(r'\bworld of tanks|мир танков|танки|танк|andoid|ios|андроид|айфон\b'), 'CAT'] = 'Игры'
data.loc[data['text'].str.contains(r'\bмоды|читы|прохождения\b'), 'CAT'] = 'Игры'
data.loc[data['text'].str.contains(r'\bмайнкрафт\b'), 'CAT'] = 'Игры'
data.loc[data['text'].str.contains(r'\bмайнкра\B'), 'CAT'] = 'Игры'
data.loc[data['text'].str.contains(r'\bplaying\b'), 'CAT'] = 'Игры'

## Animals

In [56]:
data.loc[data['text'].str.contains(r'\banimals|animal|cat|dog|cats|pets|pet|wildlife|wild\b'), 'CAT'] = 'Животные'
data.loc[data['text'].str.contains(r'\bсобака|кот|кошка|кошек|приют\b'), 'CAT'] = 'Животные'
data.loc[data['text'].str.contains(
    r'\bдомашние животные|животные|собаки|корги|чихуахуа|щенки|бульдог|хаски|хомяк\b'), 'CAT'] = 'Животные'

data.loc[data['text'].str.contains(r'\bсобак|собач|животн|млекопита|голуб\B'), 'CAT'] = 'Животные'
data.loc[data['text'].str.contains(r'\Bсобаки\b'), 'CAT'] = 'Животные'
data.loc[data['text'].str.contains(r'\bдрессиров\B'), 'CAT'] = 'Животные'

data.loc[data['text'].str.contains(r'\bжаныбар|мышык|попугай|charity\b'), 'CAT'] = 'Животные'
data.loc[data['text'].str.contains(r'\bжаныб\B'), 'CAT'] = 'Животные'

## Children

In [57]:
data.loc[data['text'].str.contains(
    r'\ba4|влад бумага|nintendo|kinder|kind|copil|bern|dziecko|лалилу|đứa trẻ\b'), 'CAT'] = 'Детское'

data.loc[data['text'].str.contains(r'\bдетск\B'), 'CAT'] = 'Детское'
data.loc[data['text'].str.contains(r'\bдетств\B'), 'CAT'] = 'Детское'
data.loc[data['text'].str.contains(r'\Bдетск\B'), 'CAT'] = 'Детское'
data.loc[data['text'].str.contains(r'\bдети\b'), 'CAT'] = 'Детское'

data.loc[data['text'].str.contains(r'\bkids|nursery|baby|kid|kinder|peppa|машинки|игрушки\b'), 'CAT'] = 'Детское'
data.loc[data['text'].str.contains(r'\bmonster|child|spiderman|superheroes|малышей\b'), 'CAT'] = 'Детское'
data.loc[data['text'].str.contains(r'\bигрушек|пеппа|princess|свинка|preschool|candy\b'), 'CAT'] = 'Детское'
data.loc[data['text'].str.contains(r'\bбарби|малыш|barbie|киндер|mario|девочка|للأطفال|الأطفال|niños|niño\b'), 'CAT'] = 'Детское'

data.loc[data['text'].str.contains(r'\bдетей\b'), 'CAT'] = 'Детское'
data.loc[data['text'].str.contains(r'\bдетей\B'), 'CAT'] = 'Детское'
data.loc[data['text'].str.contains(r'\Bдетей\B'), 'CAT'] = 'Детское'
data.loc[data['text'].str.contains(r'\bдетей\b'), 'CAT'] = 'Детское'

data.loc[data['text'].str.contains(r'\bсказки\b'), 'CAT'] = 'Детское'
data.loc[data['text'].str.contains(r'\bсказки\B'), 'CAT'] = 'Детское'
data.loc[data['text'].str.contains(r'\Bсказк\B'), 'CAT'] = 'Детское'
data.loc[data['text'].str.contains(r'\bсказк\b'), 'CAT'] = 'Детское'

data.loc[data['text'].str.contains(r'\bмультфильмы\b'), 'CAT'] = 'Детское'
data.loc[data['text'].str.contains(r'\bмультики\b'), 'CAT'] = 'Детское'
data.loc[data['text'].str.contains(r'\Bмультфильмы\b'), 'CAT'] = 'Детское'
data.loc[data['text'].str.contains(r'\bмульт\B'), 'CAT'] = 'Детское'
data.loc[data['text'].str.contains(r'\Bмульт\B'), 'CAT'] = 'Детское'
data.loc[data['text'].str.contains(r'\bмульт\b'), 'CAT'] = 'Детское'
data.loc[data['text'].str.contains(r'\bkids\b'), 'CAT'] = 'Детское'


data.loc[data['text'].str.contains(r'\bшкольник|chuyện\b'), 'CAT'] = 'Детское'
data.loc[data['text'].str.contains(r'\bшкола|школе|учеба|гдз|математика|алгебра|егэ|огэ|гдз\b'), 'CAT'] = 'Детское'
data.loc[data['text'].str.contains(r'\bschool\b'), 'CAT'] = 'Детское'
data.loc[data['text'].str.contains(r'\bшколь|математи|рисовани|нарисова|рисова\B'), 'CAT'] = 'Детское'
data.loc[data['text'].str.contains(r'\Bшкольн|chuyệ\B'), 'CAT'] = 'Детское'

data.loc[data['text'].str.contains(r'\bаниме|наруто|боруто|lego|лего\b'), 'CAT'] = 'Детское'
data.loc[data['text'].str.contains(r'\bаниме\B'), 'CAT'] = 'Детское'
data.loc[data['text'].str.contains(r'\bcartoon\b'), 'CAT'] = 'Детское'
data.loc[data['text'].str.contains(r'\bcartoon\B'), 'CAT'] = 'Детское'
data.loc[data['text'].str.contains(r'\Bcartoon\b'), 'CAT'] = 'Детское'
data.loc[data['text'].str.contains(r'\bигрушки\b'), 'CAT'] = 'Детское'

## Women

In [58]:
data.loc[data['text'].str.contains(
    r'\bцветы|цветов|квіти|гүлдер|gullar|гүлдөр|flowers|düğün\b'), 'CAT'] = 'Женские хобби'

data.loc[data['text'].str.contains(
    r'\bцвето|квіт|flower|орхиде\B'), 'CAT'] = 'Женские хобби'

data.loc[data['text'].str.contains(
    r'\bманикюр|гель|ногти|лак|лаки|лаках|гели|гелях|ресниц\b'), 'CAT'] = 'Женские хобби'

data.loc[data['text'].str.contains(
    r'\bманикюр|ногт|парикмахе|ресница|наращива|гель|нігт\B'), 'CAT'] = 'Женские хобби'

data.loc[data['text'].str.contains(
    r'\bсвадьба|wedding|hochzeit|bryllup|حفل زواج\b'), 'CAT'] = 'Женские хобби'

data.loc[data['text'].str.contains(
    r'\bсвадьба|свадебн|weddin|hochze|bryllu\B'), 'CAT'] = 'Женские хобби'

data.loc[data['text'].str.contains(r'\bпохуде\B'), 'CAT'] = 'Женские хобби'

data.loc[data['text'].str.contains(
    r'\bпохудеть|похудение|рукодельницы|похудела|новинки моды\b'), 'CAT'] = 'Женские хобби'

data.loc[data['text'].str.contains(
    r'\bfashion|fashion hacks|fitness|fashion models|biographies celebrities|все о стиле\b'), 'CAT'] = 'Женские хобби'

data.loc[data['text'].str.contains(
    r'\bфитнес|пресс|стилист|уходе за волосами|уходу за собой\b'), 'CAT'] = 'Женские хобби'

data.loc[data['text'].str.contains(
    r'\bсыночек|доченька|сыночком|крючком|для девушек|стать красивой|уход за лицом\b'), 'CAT'] = 'Женские хобби'

data.loc[data['text'].str.contains(r'\bвязан\B'), 'CAT'] = 'Женские хобби'

data.loc[data['text'].str.contains(
    r'\bкосметика|кометолог|куклы|визаж|макияж|бьюти|уход за кожей|amirxonoff\b'), 'CAT'] = 'Женские хобби'

data.loc[data['text'].str.contains(
    r'\bкосметик|косметолог|визаж|макияж|makeup|бьюти|прическ\B'), 'CAT'] = 'Женские хобби'

data.loc[data['text'].str.contains(
    r'\bmanicure|nail|cosmetics|cosmetic|makeup|cosmetology|cosmetics|cosmetic|makeup|cosmetology\b')
         , 'CAT'] = 'Женские хобби'

data.loc[data['text'].str.contains(
    r'\bjennamarbles|liza koshy|rclbeauty101|iisuperwomanii|zoella|wengie|bethany mota|lindsey stirling|\
    rosanna pansino|miranda sings\b'), 'CAT'] = 'Женские хобби'

## Shopping

In [59]:
data.loc[data['text'].str.contains(r'\bраспаковки\b'), 'CAT'] = 'Покупки'
data.loc[data['text'].str.contains(r'\bраспаковка\b'), 'CAT'] = 'Покупки'
data.loc[data['text'].str.contains(r'\Bспаковки\b'), 'CAT'] = 'Покупки'
data.loc[data['text'].str.contains(r'\bраспак|китайских товар\B'), 'CAT'] = 'Покупки'
data.loc[data['text'].str.contains(r'\baliexpress|алиэкспресс|обзоры товаров|шопинг|shopping\b'), 'CAT'] = 'Покупки'
data.loc[data['text'].str.contains(r'\bпосылок\b'), 'CAT'] = 'Покупки'
data.loc[data['text'].str.contains(r'\bпосылки\b'), 'CAT'] = 'Покупки'
data.loc[data['text'].str.contains(r'\Bпосылки\b'), 'CAT'] = 'Покупки'
data.loc[data['text'].str.contains(r'\bпосыло\B'), 'CAT'] = 'Покупки'
data.loc[data['text'].str.contains(r'\bпосылк\B'), 'CAT'] = 'Покупки'

## Religion

In [60]:
data.loc[data['text'].str.contains(r'\breligion|аллах|церква|церковь|чиркөө|church|كنيسة\b'), 'CAT'] = 'Религия'

data.loc[data['text'].str.contains(r'\bмолитвы|spiritual|القرآن|pastor\b'), 'CAT'] = 'Религия'
data.loc[data['text'].str.contains(r'\Bмолитвы|القرآن\b'), 'CAT'] = 'Религия'
data.loc[data['text'].str.contains(r'\bмолит|spiritu|القرآن\B'), 'CAT'] = 'Религия'
data.loc[data['text'].str.contains(r'\Bмолитва|القرآن\B'), 'CAT'] = 'Религия'

data.loc[data['text'].str.contains(r'\bправославия\b'), 'CAT'] = 'Религия'
data.loc[data['text'].str.contains(r'\Bправославия\b'), 'CAT'] = 'Религия'
data.loc[data['text'].str.contains(r'\bправосла\B'), 'CAT'] = 'Религия'
data.loc[data['text'].str.contains(r'\Bправосла|bible\B'), 'CAT'] = 'Религия'


data.loc[data['text'].str.contains(r'\bбога|бог|библия|божий|bible\b'), 'CAT'] = 'Религия'
data.loc[data['text'].str.contains(r'\bбиблей|bible\B'), 'CAT'] = 'Религия'

data.loc[data['text'].str.contains(r'\bбожий\B'), 'CAT'] = 'Религия'

data.loc[data['text'].str.contains(r'\bмолитвы|проповеди\b'), 'CAT'] = 'Религия'
data.loc[data['text'].str.contains(r'\bмолитвы\b'), 'CAT'] = 'Религия'
data.loc[data['text'].str.contains(r'\bмолитв|phật\B'), 'CAT'] = 'Религия'

data.loc[data['text'].str.contains(r'\bислам|намаз|намаздар|кудай|kingdom of god|christian\b'), 'CAT'] = 'Религия'

data.loc[data['text'].str.contains(r'\bислам|phật|quran|gospel\B'), 'CAT'] = 'Религия'
data.loc[data['text'].str.contains(r'\bкоран|الله|alloh|محمد|islam|phật|quran\|gospel\b'), 'CAT'] = 'Религия'

## Esoteric

In [61]:
data.loc[data['text'].str.contains(r'\bтаро|tarot|astrology\b'), 'CAT'] = 'Эзотерика'
data.loc[data['text'].str.contains(r'\Bтаро|tarot|astrology\b'), 'CAT'] = 'Эзотерика'
data.loc[data['text'].str.contains(r'\bтаро|tarot|astrolo\B'), 'CAT'] = 'Эзотерика'

data.loc[data['text'].str.contains(r'\bгадание|гадания\b'), 'CAT'] = 'Эзотерика'
data.loc[data['text'].str.contains(r'\Bгадание\b'), 'CAT'] = 'Эзотерика'
data.loc[data['text'].str.contains(r'\bгадан|ритуал\B'), 'CAT'] = 'Эзотерика'

data.loc[data['text'].str.contains(r'\bэзотерика\b'), 'CAT'] = 'Эзотерика'
data.loc[data['text'].str.contains(r'\bэзотери\B'), 'CAT'] = 'Эзотерика'
data.loc[data['text'].str.contains(r'\Bэзотерика\B'), 'CAT'] = 'Эзотерика'

data.loc[data['text'].str.contains(r'\bмагия\b'), 'CAT'] = 'Эзотерика'
data.loc[data['text'].str.contains(r'\Bмагия\b'), 'CAT'] = 'Эзотерика'
data.loc[data['text'].str.contains(r'\bмагич\B'), 'CAT'] = 'Эзотерика'

data.loc[data['text'].str.contains(r'\bритуалы|ритуал\b'), 'CAT'] = 'Эзотерика'
data.loc[data['text'].str.contains(r'\bэкстрасен|нумеролог\B'), 'CAT'] = 'Эзотерика'
data.loc[data['text'].str.contains(r'\bнумеролог\B'), 'CAT'] = 'Эзотерика'
data.loc[data['text'].str.contains(r'\bгороскоп\b'), 'CAT'] = 'Эзотерика'

data.loc[data['text'].str.contains(r'\bпаранормальные\b'), 'CAT'] = 'Эзотерика'
data.loc[data['text'].str.contains(r'\bплоской земле|плосская земля\b'), 'CAT'] = 'Эзотерика'
data.loc[data['text'].str.contains(r'\bпаранормальн\B'), 'CAT'] = 'Эзотерика'
data.loc[data['text'].str.contains(r'\bпришельцы|нло|мистика|шаман\b'), 'CAT'] = 'Эзотерика'

## Food

In [62]:
data.loc[data['text'].str.contains(r'\bкухня|먹방|mukbang|وصفات\b'), 'CAT'] = 'Кулинария'
data.loc[data['text'].str.contains(r'\Bкухня|kitchen\b'), 'CAT'] = 'Кулинария'
data.loc[data['text'].str.contains(r'\bкухн|kitchen\B'), 'CAT'] = 'Кулинария'
data.loc[data['text'].str.contains(r'\B\кухн|kitchen\B'), 'CAT'] = 'Кулинария'

data.loc[data['text'].str.contains(r'\bготовить|готовлю|eat\b'), 'CAT'] = 'Кулинария'
data.loc[data['text'].str.contains(r'\bготовит\b'), 'CAT'] = 'Кулинария'
data.loc[data['text'].str.contains(r'\bготовим\b'), 'CAT'] = 'Кулинария'

data.loc[data['text'].str.contains(r'\bвкусно\b'), 'CAT'] = 'Кулинария'
data.loc[data['text'].str.contains(r'\Bвкусно\b'), 'CAT'] = 'Кулинария'
data.loc[data['text'].str.contains(r'\bвкусн\B'), 'CAT'] = 'Кулинария'

data.loc[data['text'].str.contains(r'\bеда|món ăn\b'), 'CAT'] = 'Кулинария'

data.loc[data['text'].str.contains(r'\bкулинария\b'), 'CAT'] = 'Кулинария'
data.loc[data['text'].str.contains(r'\Bкулинар\b'), 'CAT'] = 'Кулинария'
data.loc[data['text'].str.contains(r'\bкулинар\B'), 'CAT'] = 'Кулинария'
data.loc[data['text'].str.contains(r'\B\кулинар\B'), 'CAT'] = 'Кулинария'

data.loc[data['text'].str.contains(r'\bfood|выпечке|выпечка|выпекать|выпекаю|cook|bake\b'), 'CAT'] = 'Кулинария'
data.loc[data['text'].str.contains(r'\bcooking\b'), 'CAT'] = 'Кулинария'
data.loc[data['text'].str.contains(r'\bкулинарный\b'), 'CAT'] = 'Кулинария'
data.loc[data['text'].str.contains(r'\bкулина|cook|bake\B'), 'CAT'] = 'Кулинария'

## Music

In [63]:
data.loc[data['text'].str.contains(
    r'\bmusic|topic|vevo|songs|lyrics|metal|hiphop|musica|música|müzik|guitar|reggae\b'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(
    r'\bmusic|topic|vevo|songs\B'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(
    r'\Bрmusic|topic|vevo|songs\b'), 'CAT'] = 'Музыка'


data.loc[data['text'].str.contains(
    r'\bdj|records|beats|mix|song|singersongwriter|musician|piosenki|muzică|oxxxymiron|음악\b'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(
    r'\brecords|beats|song|음악\B'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(
    r'\Brecords|beats|song|음악\b'), 'CAT'] = 'Музыка'


data.loc[data['text'].str.contains(
    r'\bsound|remix|bass|relaxin|musik|musicali|grammy|sing|muzica|сурудхои\b'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(
    r'\bsound|remix|bass|relaxin\B'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(
    r'\Bsound|remix|bass|relaxin\b'), 'CAT'] = 'Музыка'


data.loc[data['text'].str.contains(
    r'\brock|relax|disco|radio|eminem|mixtape|musics|يغني|موسيقى|musique|أغاني\b'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(
    r'\brock|relax|disco|radio|musique|أغاني\B'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(
    r'\Brock|relax|disco|radio|musique|أغاني\b'), 'CAT'] = 'Музыка'


data.loc[data['text'].str.contains(
    r'\brap|trap|hits|record|اغاني|muzyka|müzik|الاغاني|şarkı|şarkılar\b'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(
    r'\btrap|hits|record|اغاني|muzyka|müzik|الاغاني|şarkı|piosen\B'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(
    r'\Btrap|hits|record|اغاني|muzyka|müzik|الاغاني\b'), 'CAT'] = 'Музыка'

data.loc[data['text'].str.contains(
    r'\bhindi|videoclips|musical\b'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(
    r'\bhindi|videoclips|musical\B'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(
    r'\Bhindi|videoclips|musical\b'), 'CAT'] = 'Музыка'

data.loc[data['text'].str.contains(
    r'\blisten|label|audio|cover\b'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(
    r'\blisten|label|audio|cover\B'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(
    r'\Blisten|label|audio|cover\b'), 'CAT'] = 'Музыка'


data.loc[data['text'].str.contains(
    r'\bpiano|jazz|vocal|clips|songwriter|nhạc\b'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(
    r'\bpiano|jazz|vocal|clips|songwriter|nhạc\B'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(
    r'\Bpiano|jazz|vocal|clips|songwriter|nhạc\b'), 'CAT'] = 'Музыка'

In [64]:
data.loc[data['text'].str.contains(
    r'\bmusic|remix|shnurov\b'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(r'\Bmusic\b'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(r'\bmusic\B'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(r'\Bmusic\B'), 'CAT'] = 'Музыка'

data.loc[data['text'].str.contains(r'\brecords\b'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(r'\Brecords\b'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(r'\brecords\B'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(r'\Brecords\B'), 'CAT'] = 'Музыка'

data.loc[data['text'].str.contains(
    r'\bмузыка|qoshiqlari|әндер\b'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(r'\bмузыки|рэп\b'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(r'\Bмузык\b'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(r'\bмузык|qoshiql\B'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(r'\Bмузык\B'), 'CAT'] = 'Музыка'

data.loc[data['text'].str.contains(r'\bдискотека\b'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(r'\Bдискотека\b'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(r'\bдискот\B'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(r'\Bдискот\B'), 'CAT'] = 'Музыка'

data.loc[data['text'].str.contains(r'\bшансон\b'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(r'\Bшансон\b'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(r'\bшансон\B'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(r'\Bшансон\B'), 'CAT'] = 'Музыка'

data.loc[data['text'].str.contains(
    r'\bкаверы|караоке|rizanovauz|сектор газа\b'), 'CAT'] = 'Музыка'

data.loc[data['text'].str.contains(r'\ddance|аудио\b'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(r'\bклипы\b'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(r'\bмузыкальные\b'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(r'\bsong|треки\b'), 'CAT'] = 'Музыка'

data.loc[data['text'].str.contains(r'\bпесни\b'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(r'\bsound\b'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(r'\bsound\B'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(r'\bпесня\b'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(r'\Bпесни\b'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(r'\bпесн\B'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(r'\Bпесн\B'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(r'\bпевицы\b'), 'CAT'] = 'Музыка'

data.loc[data['text'].str.contains(r'\bгитара\b'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(r'\Bгитара\b'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(r'\bгитар\B'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(r'\Bгитар\B'), 'CAT'] = 'Музыка'

data.loc[data['text'].str.contains(r'\btopic\b'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(r'\bпианино\b'), 'CAT'] = 'Музыка'

data.loc[data['text'].str.contains(r'\bvevo\b'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(r'\Bvevo\b'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(r'\bvevo\B'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(r'\Bvevo\B'), 'CAT'] = 'Музыка'

data.loc[data['text'].str.contains(r'\bvevo\b'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(r'\Bvevo\b'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(r'\bvevo\B'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(r'\Bvevo\B'), 'CAT'] = 'Музыка'

data.loc[data['text'].str.contains(r'\brelaxing\b'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(r'\brelax\b'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(r'\brelax\B'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(r'\bbeats\b'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(r'\bпевец\b'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(r'\brock\b'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(r'\bradio\b'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(r'\bрадио\b'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(r'\bрадио\B'), 'CAT'] = 'Музыка'
data.loc[data['text'].str.contains(r'\Bрадио\b'), 'CAT'] = 'Музыка'

## Casino

In [65]:
data.loc[data['text'].str.contains(
    r'\bcasino|slot|slot machine|slot play|blackjack\b'), 'CAT'] = 'Казино'
data.loc[data['text'].str.contains(r'\Bcasino\b'), 'CAT'] = 'Казино'
data.loc[data['text'].str.contains(r'\bcasino\B'), 'CAT'] = 'Казино'
data.loc[data['text'].str.contains(r'\Bcasino\B'), 'CAT'] = 'Казино'

data.loc[data['text'].str.contains(
    r'\bказино|poker|покер\b'), 'CAT'] = 'Казино'
data.loc[data['text'].str.contains(r'\Bказино\b'), 'CAT'] = 'Казино'
data.loc[data['text'].str.contains(r'\bказино\B'), 'CAT'] = 'Казино'
data.loc[data['text'].str.contains(r'\Bказино\B'), 'CAT'] = 'Казино'

data.loc[data['text'].str.contains(r'\bставки\b'), 'CAT'] = 'Казино'

data.loc[data['text'].str.contains(
    r'\bигровые автоматы|игровой автомат\b'), 'CAT'] = 'Казино'
data.loc[data['text'].str.contains(r'\bлото|прогнозы\b'), 'CAT'] = 'Казино'

## Hobbiess

In [66]:
data.loc[data['text'].str.contains(
    r'\bкомбайн|сеем|посев|урожай|агрономический|агро|camping|truck|coin|workout|bodybuilder\b'), 'CAT'] = 'Мужские Хобби'
data.loc[data['text'].str.contains(
    r'\bагро|ферме|camping|coin\B'), 'CAT'] = 'Мужские Хобби'

data.loc[data['text'].str.contains(
    r'\bогоро|wrestle\B'), 'CAT'] = 'Мужские Хобби'

data.loc[data['text'].str.contains(
    r'\bsurvival|boxing|science|meat|aviation|tuning|moto|hockey|farmer|diving|barber|gordon ramsay\b')
         , 'CAT'] = 'Мужские Хобби'
data.loc[data['text'].str.contains(
    r'\bохота|woodworking|мир танков|танки|militer|cossacks|казаки|bounty hunter\b'), 'CAT'] = 'Мужские Хобби'
data.loc[data['text'].str.contains(
    r'\Bохота|militer|truck\b'), 'CAT'] = 'Мужские Хобби'
data.loc[data['text'].str.contains(
    r'\bохот|militer|truck\B'), 'CAT'] = 'Мужские Хобби'

data.loc[data['text'].str.contains(
    r'\bадвокат|юрист|закон|полиция|гидравлический\b'), 'CAT'] = 'Мужские Хобби'
data.loc[data['text'].str.contains(
    r'\bадвокат|военнослу|ветера\B'), 'CAT'] = 'Мужские Хобби'

data.loc[data['text'].str.contains(
    r'\bfootball|săn\b'), 'CAT'] = 'Мужские Хобби'
data.loc[data['text'].str.contains(r'\bфутбол\b'), 'CAT'] = 'Мужские Хобби'
data.loc[data['text'].str.contains(r'\bфутбол|săn\B'), 'CAT'] = 'Мужские Хобби'

data.loc[data['text'].str.contains(
    r'\bтюнинг|ваз|лада|тонировка|автообзор\b'), 'CAT'] = 'Мужские Хобби'
data.loc[data['text'].str.contains(r'\bфутбол\b'), 'CAT'] = 'Мужские Хобби'
data.loc[data['text'].str.contains(r'\bтюнинг\B'), 'CAT'] = 'Мужские Хобби'

data.loc[data['text'].str.contains(
    r'\bрыбалка|теннис|сварка|сварке|монтажник|водитель|такси|mafia|камаз|fishing|fish\b'), 'CAT'] = 'Мужские Хобби'
data.loc[data['text'].str.contains(
    r'\bрыбалке|погрузчик|экскаватор\b'), 'CAT'] = 'Мужские Хобби'
data.loc[data['text'].str.contains(r'\Bрыбалка\b'), 'CAT'] = 'Мужские Хобби'
data.loc[data['text'].str.contains(r'\bрыба\B'), 'CAT'] = 'Мужские Хобби'
data.loc[data['text'].str.contains(r'\Bрыба\B'), 'CAT'] = 'Мужские Хобби'

data.loc[data['text'].str.contains(
    r'\bгараж|garage|грузовик|трактор\b'), 'CAT'] = 'Мужские Хобби'
data.loc[data['text'].str.contains(r'\Bгараж\b'), 'CAT'] = 'Мужские Хобби'
data.loc[data['text'].str.contains(
    r'\bгараж|грузовик|трактор\B'), 'CAT'] = 'Мужские Хобби'
data.loc[data['text'].str.contains(r'\Bгараж\B'), 'CAT'] = 'Мужские Хобби'

data.loc[data['text'].str.contains(r'\bавто\b'), 'CAT'] = 'Мужские Хобби'
data.loc[data['text'].str.contains(r'\Bавто\b'), 'CAT'] = 'Мужские Хобби'
data.loc[data['text'].str.contains(r'\bавтомо\B'), 'CAT'] = 'Мужские Хобби'
data.loc[data['text'].str.contains(r'\bмотоцикл\b'), 'CAT'] = 'Мужские Хобби'
data.loc[data['text'].str.contains(r'\bмотоцикл\B'), 'CAT'] = 'Мужские Хобби'

data.loc[data['text'].str.contains(
    r'\bбарбер|fighters|бойцы\b'), 'CAT'] = 'Мужские Хобби'

data.loc[data['text'].str.contains(
    r'\bстанок|казак\b'), 'CAT'] = 'Мужские Хобби'
data.loc[data['text'].str.contains(r'\bремонт\b'), 'CAT'] = 'Мужские Хобби'
data.loc[data['text'].str.contains(r'\Bремонт\b'), 'CAT'] = 'Мужские Хобби'
data.loc[data['text'].str.contains(r'\bремонт\B'), 'CAT'] = 'Мужские Хобби'
data.loc[data['text'].str.contains(r'\Bремонт\B'), 'CAT'] = 'Мужские Хобби'

data.loc[data['text'].str.contains(
    r'\bстроительство\b'), 'CAT'] = 'Мужские Хобби'
data.loc[data['text'].str.contains(r'\bстроитель\b'), 'CAT'] = 'Мужские Хобби'
data.loc[data['text'].str.contains(
    r'\Bстроительство\b'), 'CAT'] = 'Мужские Хобби'
data.loc[data['text'].str.contains(r'\bстроите\B'), 'CAT'] = 'Мужские Хобби'

data.loc[data['text'].str.contains(
    r'\bинструменты\b'), 'CAT'] = 'Мужские Хобби'
data.loc[data['text'].str.contains(r'\bинструмент\b'), 'CAT'] = 'Мужские Хобби'
data.loc[data['text'].str.contains(
    r'\Bинструменты\b'), 'CAT'] = 'Мужские Хобби'
data.loc[data['text'].str.contains(
    r'\bинструмент|барбер\B'), 'CAT'] = 'Мужские Хобби'
data.loc[data['text'].str.contains(r'\Bинструмент\B'), 'CAT'] = 'Мужские Хобби'

data.loc[data['text'].str.contains(
    r'\bархеологический|археолог|археология|гбдд|战争\b'), 'CAT'] = 'Мужские Хобби'
data.loc[data['text'].str.contains(r'\Bархеолог\b'), 'CAT'] = 'Мужские Хобби'
data.loc[data['text'].str.contains(r'\bархеолог\B'), 'CAT'] = 'Мужские Хобби'
data.loc[data['text'].str.contains(r'\bподзем\B'), 'CAT'] = 'Мужские Хобби'

data.loc[data['text'].str.contains(r'\bгаи\b'), 'CAT'] = 'Мужские Хобби'

data.loc[data['text']
         .str.contains(r'\bпоходы|лес|горы|ножы|оружие|армия|солдат|война|войны|ссср|история|\
         вторая мировая|military|авторынок\b'), 'CAT'] = 'Мужские Хобби'
data.loc[data['text'].str.contains(
    r'\bисторич|military\B'), 'CAT'] = 'Мужские Хобби'
data.loc[data['text'].str.contains(r'\bисторич\B'), 'CAT'] = 'Мужские Хобби'
data.loc[data['text'].str.contains(
    r'\bтуризм|historium\b'), 'CAT'] = 'Мужские Хобби'
data.loc[data['text'].str.contains(r'\bтуриз\B'), 'CAT'] = 'Мужские Хобби'
data.loc[data['text'].str.contains(r'\bтурис\B'), 'CAT'] = 'Мужские Хобби'
data.loc[data['text'].str.contains(r'\bтуриc\B'), 'CAT'] = 'Мужские Хобби'
data.loc[data['text'].str.contains(r'\bвыживание\B'), 'CAT'] = 'Мужские Хобби'

data.loc[data['text'].str.contains(
    r'\bнаходки|клад|металлоискатель|магнит|монеты|аукцион|ретро|золото|золоте|золотых|сталк\b'), 'CAT'] = 'Мужские Хобби'
data.loc[data['text'].str.contains(r'\bметал\B'), 'CAT'] = 'Мужские Хобби'
data.loc[data['text'].str.contains(r'\bклад|мужск\B'), 'CAT'] = 'Мужские Хобби'
data.loc[data['text'].str.contains(r'\bантиква\B'), 'CAT'] = 'Мужские Хобби'
data.loc[data['text'].str.contains(r'\bдальнобой\b'), 'CAT'] = 'Мужские Хобби'
data.loc[data['text'].str.contains(r'\bдальнобой\B'), 'CAT'] = 'Мужские Хобби'
data.loc[data['text'].str.contains(r'\bмото\B'), 'CAT'] = 'Мужские Хобби'

data.loc[data['text'].str.contains(
    r'\bwojskowy|militær|askeri|الجيش|صيد السمك|fiskeri|angeln|reparatur|repair|军事\b'), 'CAT'] = 'Мужские Хобби'
data.loc[data['text'].str.contains(
    r'\bmilitä|askeri|fiskeri|reparatu|reparati|avlanm\B'), 'CAT'] = 'Мужские Хобби'

data.loc[data['text'].str.contains(
    r'\bhunting|jagt|الصيد|jagd|avlanma|soldat|جندي\b'), 'CAT'] = 'Мужские Хобби'


data.loc[data['text']
         .str.contains(r'\bтехнологии|наука|изобретения|штукатурка|автомобили|racer nascar\b'), 'CAT'] = 'Мужские Хобби'

## Movies&TVseries

In [67]:
data.loc[data['text'].str.contains(
    r'\bmovies|movie|films|trailers|cinema|türkiyenin|filmmaker|bollywood|osman|hollywood|documentary|\
    cinéma|filming|filmer\B'), 'CAT'] = 'Кино и Сериалы'
data.loc[data['text'].str.contains(
    r'\bmovies|movie|films|trailers|cinema|türkiyenin|filmmaker|bollywood|osman|hollywood|documentary|\
    cinéma|filming|filmer\b'), 'CAT'] = 'Кино и Сериалы'
data.loc[data['text'].str.contains(
    r'\Bmovies|movie|films|trailers|cinema|türkiyenin|filmmaker|bollywood|osman|hollywood|documentary|\
    cinéma|filming|filmer\b'), 'CAT'] = 'Кино и Сериалы'

In [68]:
data.loc[data['text'].str.contains(
    r'\Bfilm|فيلم|netflix|phim ảnh|دراما\B'), 'CAT'] = 'Кино и Сериалы'
data.loc[data['text'].str.contains(
    r'\bfilm|netflix|phim ảnh|دراما\B'), 'CAT'] = 'Кино и Сериалы'
data.loc[data['text'].str.contains(
    r'\bfilm|netflix|bbc|hbo|phim ảnh|دراما\b'), 'CAT'] = 'Кино и Сериалы'
data.loc[data['text'].str.contains(
    r'\Bfilm|netflix|phim ảnh|دراما\b'), 'CAT'] = 'Кино и Сериалы'

data.loc[data['text'].str.contains(r'\Bmovies\B'), 'CAT'] = 'Кино и Сериалы'
data.loc[data['text'].str.contains(r'\Bmovi\B'), 'CAT'] = 'Кино и Сериалы'
data.loc[data['text'].str.contains(r'\bmovi\B'), 'CAT'] = 'Кино и Сериалы'
data.loc[data['text'].str.contains(
    r'\bmovies|电影解说\b'), 'CAT'] = 'Кино и Сериалы'
data.loc[data['text'].str.contains(r'\bmovi\b'), 'CAT'] = 'Кино и Сериалы'
data.loc[data['text'].str.contains(r'\Bmovies\b'), 'CAT'] = 'Кино и Сериалы'
data.loc[data['text'].str.contains(r'\Bmovi\b'), 'CAT'] = 'Кино и Сериалы'


data.loc[data['text'].str.contains(
    r'\bкино|телешоу\B'), 'CAT'] = 'Кино и Сериалы'
data.loc[data['text'].str.contains(
    r'\bкино|военные|jamoasi|kurtlar vadisi pusu|فيلم|долина волков|电影\b'), 'CAT'] = 'Кино и Сериалы'
data.loc[data['text'].str.contains(r'\Bкино|电影\b'), 'CAT'] = 'Кино и Сериалы'
data.loc[data['text'].str.contains(
    r'\Bкино|cinema|电影\B'), 'CAT'] = 'Кино и Сериалы'

data.loc[data['text'].str.contains(r'\bkino\B'), 'CAT'] = 'Кино и Сериалы'
data.loc[data['text'].str.contains(r'\bkino|영화를\b'), 'CAT'] = 'Кино и Сериалы'
data.loc[data['text'].str.contains(r'\bkino\B'), 'CAT'] = 'Кино и Сериалы'
data.loc[data['text'].str.contains(r'\Bkino\b'), 'CAT'] = 'Кино и Сериалы'

data.loc[data['text'].str.contains(
    r'\bсериал|dizilerini\B'), 'CAT'] = 'Кино и Сериалы'
data.loc[data['text'].str.contains(
    r'\bсериал|i̇stanbulun|türkiyenin|osman|TV dizisi|dizisi|televizyon|中国电视剧\b'), 'CAT'] = 'Кино и Сериалы'
data.loc[data['text'].str.contains(
    r'\bсериалы|дорамы|avaz oxun|영화리뷰\b'), 'CAT'] = 'Кино и Сериалы'
data.loc[data['text'].str.contains(
    r'\bсеріал|dizil\B'), 'CAT'] = 'Кино и Сериалы'

data.loc[data['text'].str.contains(r'\bфильм\B'), 'CAT'] = 'Кино и Сериалы'
data.loc[data['text'].str.contains(
    r'\bфильмов|documentaries|zawaj maslaha\b'), 'CAT'] = 'Кино и Сериалы'
data.loc[data['text'].str.contains(r'\bфильм\b'), 'CAT'] = 'Кино и Сериалы'

data.loc[data['text'].str.contains(r'\bcomedy\B'), 'CAT'] = 'Кино и Сериалы'
data.loc[data['text'].str.contains(r'\bcomedy\b'), 'CAT'] = 'Кино и Сериалы'
data.loc[data['text'].str.contains(r'\bcomedy\B'), 'CAT'] = 'Кино и Сериалы'
data.loc[data['text'].str.contains(r'\Bcomedy\b'), 'CAT'] = 'Кино и Сериалы'


data.loc[data['text'].str.contains(r'\bдетектив\B'), 'CAT'] = 'Кино и Сериалы'
data.loc[data['text'].str.contains(
    r'\bдетектив|nollywood\b'), 'CAT'] = 'Кино и Сериалы'
data.loc[data['text'].str.contains(r'\bдетект\B'), 'CAT'] = 'Кино и Сериалы'
data.loc[data['text'].str.contains(r'\Bдетект\b'), 'CAT'] = 'Кино и Сериалы'

data.loc[data['text'].str.contains(
    r'\bшоу|cinema\B'), 'CAT'] = 'Кино и Сериалы'
data.loc[data['text'].str.contains(r'\bшоу\b'), 'CAT'] = 'Кино и Сериалы'
data.loc[data['text'].str.contains(r'\bшоу\B'), 'CAT'] = 'Кино и Сериалы'
data.loc[data['text'].str.contains(
    r'\Bшоу|cinema\b'), 'CAT'] = 'Кино и Сериалы'
data.loc[data['text'].str.contains(r'\bмарвел\b'), 'CAT'] = 'Кино и Сериалы'
data.loc[data['text'].str.contains(
    r'\bтрейлер|трейлеры\b'), 'CAT'] = 'Кино и Сериалы'

data.loc[data['text'].str.contains(
    r'\bкомаров|мир наизнанку|званый ужин|дом2|дом 2|красавица и чудовище\b'), 'CAT'] = 'Кино и Сериалы'

data.loc[data['text'].str.contains(
    r'\bмелодрамы|مسلسل تركي|turkish series|مسلسل تلفزيونى|tv series|xena warrior princess|xena|드라마\b')
         , 'CAT'] = 'Кино и Сериалы'

data.loc[data['text'].str.contains(
    r'\bмелод|tv seri\B'), 'CAT'] = 'Кино и Сериалы'

data.loc[data['text']
         .str.contains(r'\bквартал 95|wwe|дом-2|kin4k|уральские пельмени|central partnership|دراما\b')
         , 'CAT'] = 'Кино и Сериалы'

## Sports&Hobbiess

In [69]:
data.loc[data['text'].str.contains(
    r'\bwwe|how ridiculous|thats amazing shorts|mlg highlights|one championship|formula 1|nfl|highlight heaven|\
    fifatv|wimbledon|espn fans|clutchpoints|carson hockey|ufc  ultimate fighting championsh|kplus sports|\
    sky sports football|sports mix|dude perfect|thats why mma|neom|espn|rojitohd|tudn méxico|bt sport|bleacher report\
    |tnt sports brasil|jovem pan esportes|house of highlights|матч тв|ekattor tv|abema 格闘ch公式|t10 league official|\
    blacktiph|oliver nordin|fc barcelona|modern goalkeeping|olympics|espn brasil|sports central|icc|ashstudio7|\
    ontime sports|serie a|cricketcomau|news24 sports|dallmyd|big ideas in under a minute|jogo aberto|sports tak|\
    s sport|liverpool fc|futparódias|cracks|bwf tv|steven strangles people|hindi ar entertainments|elias dosunmu|\
    hooper highlights|thats amazing|nbc sports|aakash chopra|robinson foundry|fighthypecom|whats inside family|\
    score 90|kora plus|supersport|strops|동현kldh|rick azas|bein sports france|alex segura lr  canal secundario|\
    alex drachnik|krav maga caveira  mestre wesley |power volleyball|us open tennis championships|motogp|이스타tv|\
    pamela reif|álex segura lr|rizin fighting federation|pakistan cricket|the united stand|\
    england  wales cricket board|kingswish|f2freestylers  ultimate soccer sk|dazn boxing|top rank boxing|\
    kabaddi365com|demolitionranch|moalifc|ivan rusakov|卓キチちゃんねる|fr4nce|spor delisi hd|laliga santander|\
    sports direct|skip and shannon undisputed|dazn japan\b'), 'CAT'] = 'Спорт'

In [70]:
data.loc[data['text'].str.contains(
    r'\bмма|mma|futbol|ufc|كرة القدم|cricket|nba\b'), 'CAT'] = 'Спорт'
data.loc[data['text'].str.contains(r'\bбокс|боёв|боев\b'), 'CAT'] = 'Спорт'
data.loc[data['text'].str.contains(
    r'\bбокс|трениров|кроссфи\B'), 'CAT'] = 'Спорт'
data.loc[data['text'].str.contains(r'\bбоев|боёв\B'), 'CAT'] = 'Спорт'

data.loc[data['text'].str.contains(r'\bспорт\b'), 'CAT'] = 'Спорт'
data.loc[data['text'].str.contains(r'\Bспорт\b'), 'CAT'] = 'Спорт'
data.loc[data['text'].str.contains(r'\bспорт\B'), 'CAT'] = 'Спорт'
data.loc[data['text'].str.contains(r'\Bспорт\B'), 'CAT'] = 'Спорт'

data.loc[data['text'].str.contains(
    r'\bфутбол|тенис|теннис|баскетбол|football|хоккей|гольф|бильярд|регби|rugby|tennis\b'), 'CAT'] = 'Спорт'
data.loc[data['text'].str.contains(
    r'\Bфутбол|football|basketball\b'), 'CAT'] = 'Спорт'
data.loc[data['text'].str.contains(
    r'\bфутбол|football|basketba\B'), 'CAT'] = 'Спорт'
data.loc[data['text'].str.contains(
    r'\Bфутбол|хоккей|баскетбол|basketball\B'), 'CAT'] = 'Спорт'

data.loc[data['text'].str.contains(r'\bsport\b'), 'CAT'] = 'Спорт'
data.loc[data['text'].str.contains(r'\bsports\b'), 'CAT'] = 'Спорт'
data.loc[data['text'].str.contains(r'\Bsport\b'), 'CAT'] = 'Спорт'
data.loc[data['text'].str.contains(r'\Bsports\b'), 'CAT'] = 'Спорт'
data.loc[data['text'].str.contains(r'\bsport\B'), 'CAT'] = 'Спорт'
data.loc[data['text'].str.contains(r'\Bsport\B'), 'CAT'] = 'Спорт'

data.loc[data['text'].str.contains(r'\bgym\b'), 'CAT'] = 'Спорт'
data.loc[data['text'].str.contains(r'\Bgym\b'), 'CAT'] = 'Спорт'
data.loc[data['text'].str.contains(r'\bgym\B'), 'CAT'] = 'Спорт'
data.loc[data['text'].str.contains(r'\Bgym\B'), 'CAT'] = 'Спорт'

data.loc[data['text'].str.contains(
    r'\bnavi|team liquid|psglgd|evil geniuses|team spirit|team secret|newbee|virtuspro|vici gaming|\
    wings gaming|astralis|invictus gaming|natus vincere|t1|geng esports|alliance|tnc predator|ehome|\
    faze clan|sk gaming|cdec gaming|san francisco shock|ninjas in pyjamas|lgdforever young|royal never give up|\
    gambit esports|g2 esports|mvp phoenix|samsung galaxy|edward gaming|shanghai dragons|team vitality|\
    cloud9|mineski|team envy|fpx esports|team empire|heroic|dwg kia|mibr|team dk|london spitfire|\
    furia esports|ence|susquehanna soniqs|complexity gaming|infuesports|optic gaming|new york excelsior|\
    mvp black|spacestation gaming|seoul dynasty|team dignitas|team aster|samsung galaxy white|\
    four angry men|quincy crew\b'), 'CAT'] = 'Киберспорт'

## Media

In [71]:
data.loc[data['text'].str.contains(
    r'\bsocialism|democrat|politics|republican|debate|سياسة|سیاست|politik|polityka|politică|putin|\
    trump|biden\b'), 'CAT'] = 'Новости и политика'

data.loc[data['text'].str.contains(
    r'\bтелеком|телерад|кремл|yangiliklar\B'), 'CAT'] = 'Новости и политика'

data.loc[data['text'].str.contains(
    r'\bmedia|political|reportages|федеральный канал\b'), 'CAT'] = 'Новости и политика'
data.loc[data['text'].str.contains(r'\Bmedia\b'), 'CAT'] = 'Новости и политика'

data.loc[data['text'].str.contains(
    r'\Bмедиа|жанылыктар|tin tuc|tin tức|영화\B'), 'CAT'] = 'Новости и политика'
data.loc[data['text'].str.contains(
    r'\bмедиа|жанылыктар|tin tuc|tin tức|영화\B'), 'CAT'] = 'Новости и политика'
data.loc[data['text'].str.contains(
    r'\Bмедиа|жанылыктар|tin tuc|tin tức|영화\b'), 'CAT'] = 'Новости и политика'
data.loc[data['text'].str.contains(
    r'\bмедиа|жанылыктар|tin tuc|tin tức|영화\b'), 'CAT'] = 'Новости и политика'

data.loc[data['text'].str.contains(
    r'\bтелеканал|خبر|뉴스\b'), 'CAT'] = 'Новости и политика'

data.loc[data['text'].str.contains(
    r'\bnews|television|nyheder|știri|nachrichten|путина\b'), 'CAT'] = 'Новости и политика'
data.loc[data['text'].str.contains(
    r'\Bnews|nachrichten\b'), 'CAT'] = 'Новости и политика'
data.loc[data['text'].str.contains(
    r'\bnews|nachrichten\B'), 'CAT'] = 'Новости и политика'

data.loc[data['text'].str.contains(
    r'\bновости|новостей|жаңылыктар|жаңылыктар|pressa|ахбор|янгилик|сми|خبری\b'), 'CAT'] = 'Новости и политика'
data.loc[data['text'].str.contains(
    r'\bтелеканал|shokuz|uzmigrant|journalist\b'), 'CAT'] = 'Новости и политика'
data.loc[data['text'].str.contains(
    r'\bновини\b'), 'CAT'] = 'Новости и политика'
data.loc[data['text'].str.contains(
    r'\Bновости|янгиликb'), 'CAT'] = 'Новости и политика'
data.loc[data['text'].str.contains(
    r'\bновос|xəbər\B'), 'CAT'] = 'Новости и политика'
data.loc[data['text'].str.contains(r'\Bновос\B'), 'CAT'] = 'Новости и политика'
data.loc[data['text'].str.contains(
    r'\byangik|янгилик\B'), 'CAT'] = 'Новости и политика'

data.loc[data['text'].str.contains(
    r'\bпрямом эфире|прямой эфир|新闻\b'), 'CAT'] = 'Новости и политика'


data.loc[data['text'].str.contains(
    r'\Bполитик\B'), 'CAT'] = 'Новости и политика'
data.loc[data['text'].str.contains(
    r'\bполитик\B'), 'CAT'] = 'Новости и политика'
data.loc[data['text'].str.contains(
    r'\Bполитика\b'), 'CAT'] = 'Новости и политика'
data.loc[data['text'].str.contains(
    r'\bполитика|رسانه\b'), 'CAT'] = 'Новости и политика'
data.loc[data['text'].str.contains(
    r'\bпутин|اخبار\b'), 'CAT'] = 'Новости и политика'

data.loc[data['text'].str.contains(
    r'\bтелека\B'), 'CAT'] = 'Новости и политика'
data.loc[data['text'].str.contains(
    r'\bновостной|газета\b'), 'CAT'] = 'Новости и политика'
data.loc[data['text'].str.contains(
    r'\bжурналиста\b'), 'CAT'] = 'Новости и политика'
data.loc[data['text'].str.contains(
    r'\bжурналис\B'), 'CAT'] = 'Новости и политика'
data.loc[data['text'].str.contains(
    r'\bдепутат\B'), 'CAT'] = 'Новости и политика'

data.loc[data['text']
         .str.contains(r'\bнтв|россия 1|стс|первый канал|пятый канал россия|zamin uz|рен тв|\
         صوت بيروت\b'), 'CAT'] = 'Новости и политика'

data.loc[data['text'].str.contains(
    r'\bone plus one|meet the press|the sunday project|630 with george negus|weekend today|\
    a current affair|living black|weekend breakfast|four corners\b'), 'CAT'] = 'Новости и политика'

## Overriding

In [72]:
data.loc[data['text'].str.contains(
    r'\bviralhog|jayprehistoricpets|the dodo|corbin maxey|cute zone|animais da fazenda|もちまる日記|\
    the cincinnati zoo  botanical|topi the corgi|wow animals|rottweiler life|sbs tv동물농장x애니멀봐|\
    fish cutting|gacha omnia|badri narayan bhadra|tucker budzyn|life with klee kai|chat drôle|\
    the soy루퐁이네|life of sammie  summer|come and enjoy|the secret life of my hamster|aty|dehakims aviary|\
    billy smith|dog rescue shelter mladenovac|alshad ahmad|dodo kids|funnyplox|祁丽英 ying|brave wilderness|\
    owlkitty|kritter klub|bbc earth|doug the pug|max the husky|aaronsanimals|wilderness tv|dehakims|\
    кошка хлоя  котята|maymo|saveafox|toegrips|tamil galatta news|dean schneider|animal aid unlimited|\
    sarcastic nitin|مرتقون |tofu the corgi|sarpmitra akash jadhav|клондайктв неверо|tyler nolan|latest sightings|\
    the hoof gp|panji petualang|おちゃんねる|cute_pet_basic_state|sixty formula|tatyana burdina|ginghart|\
    dogs are the best pets|the pet collective|wildlife channel|onemoreplease|rapid liquid|ちゃんねる鰐|\
    cp wild lanka|kucing om wepe|gernot koinig|rumble viral|keyush the stunt dog|感動猫動画|cutest animals|\
    elephant thailand|정브르|petco|louie the beagle|animals ct|pitbull boyka|iqbalizda|kakoas world|\
    bengkel kicau|크림히어로즈|afinatv|victor brasil|다흑|моя анимированная|bondi vet|タイピー日記taipi|\
    tám tới tấp|pecoチャンネルさん【癒や|사모예드 티코|grumpy dogs|the world of cats|husmut|fressnapf|\
    beagle universe|고양이산책|funny pet videos|시바견 곰이탱이여우\b'), 'CAT'] = 'Животные'

data.loc[data['text'].str.contains(
    r'\blankybox|ffuntv|gaurav arora vlogs|shurikworld|billy karavdic|mikecrack|aphmau|invictor|\
    ssundee|cocohindiyt|dc studio|mrbeast gaming|robin hood gamer|onlyjayus|bagas craft|itsfunneh|\
    markiplier|rb playing|league of legends|ign|xzit thamer|fgteev|techno gamerz|karim juega|\
    davidjustinn|sambucha|alexa real|dimucc|degoboom|lstoast|total gaming|frostbite gaming|patife|\
    mati spano|まいぜんシスターズ|beluga|ghs|fixeye|minecraft parodileri|zbing z|communitygame|\
    鬥士工作室 fighter studio hk|mobile legends bang bang official|coryxkenshin|aboflah|steveee|\
    евгенбро|slenderman™|lispyleaf|edisonpts|craftee|شبكة العاب العرب |beniju|luky|frost diamond|\
    prestonplayz|hindustan gamer loggy|windah basudara|morealia|deonade|miawaug|e-mastersensei|\
    idolatro24|caylus|lyna|free fire india official|slogo|angry birds|flamingo|chapati hindustani gamer|\
    anthonysenpai|eltrollino|as gaming|lowgrade|kuplinov play|tsgeditz|auron|dang matt smith|\
    whengamersfail|pathofail|restorer tv|geleia|julia minegirl|ferited|gamingwithkev|gevids|\
    mythpat|kindly keyin|sublimator|mều channel|pratama eriz|antonio palmucci|tor torpedo|\
    homeanimations|поззи|checkpoint|two side gamers|캐릭온 tv\b'), 'CAT'] = 'Игры'

data.loc[data['text'].str.contains(
    r'\bcocomelon|like nastya|elis and thomas tube|vlad and niki|kids diana show|toys and colors|\
    d billions|pinkfong baby shark|el reino infantil|voot kids|vlad và nikita|genevieves playhouse|\
    looloo kids|chuchu tv nursery rhymes|el payaso plim plim|ryans world|super jojo|masha and the bear|\
    peppa pig|wolfoo family|la granja de zenón|like nastya vnm|wolfoos stories|wolfoo channel|\
    bounce patrol|infobells|व्लाद और निक|come play with me|totoy kids|маша и медведь|babybus|nick jr|\
    luccas neto|blippi|diana and roma esp|vlad and niki arabic|lego|kids roma show|the mik maks|\
    kinder spielzeug kanal|moonbug kids|like nastya esp|vania mania kids|rjkarishma|влад и никита|\
    mundo bita|infobells bangla|little baby bum|vlad and niki ara|little angel: nursery rhymes|हिंदी|\
    boom buddies|babybus|galinha pintadinha|like nastya ae|masha y el oso|wb kids|luli pampín|\
    wolfoo tiếng việt|toycantando|disney junior|moonbug kids|vlad y nikita show|little angel bahasa indonesia|\
    jorge el curioso en español|karim juega|vlad and niki idn|like nastya prt|steve and maggie|\
    kids channel india|diana and roma ind|infobells|osratouna tv|super jojo|um herói do coração|\
    oddbods|moonbug kids|vlad ve nikita|wolfoo and friends|blippi toys|cvs 3d rhymes & kids songs|ماشا و الدب|\
    o reino infantil|talking tom|vlad and niki prt|infobells|oyuncakoynuyorum|diana and roma vnm|\
    canciones infantiles|marah kg|famosa international|blippi español|sesame street|little world|\
    babybus|mattel español|super crazy kids|johny familyshow|левий лев|dreamworks madagascar\b'), 'CAT'] = 'Детское'

In [73]:
data.loc[data['text'].str.contains(
    r'\bзаработок в интернете|пассивный доход|фондовый рынок\b'), 'CAT'] = 'Финансы'

data.loc[data['text'].str.contains(
    r'\bкинолог|дрессировкой|дрессировка\b'), 'CAT'] = 'Животные'
data.loc[data['text'].str.contains(r'\bдрессиров\B'), 'CAT'] = 'Животные'

data.loc[data['text'].str.contains(
    r'\bretriev|shepher|bulldo|terrie|poodl\B'), 'CAT'] = 'Животные'
data.loc[data['text'].str.contains(
    r'\blabrador retrievers|german shepherds|golden retrievers|french bulldogs|bulldogs|beagles|\
    poodles|rottweilers|german shorthaired pointers|yorkshire terriers\b'), 'CAT'] = 'Животные'

data.loc[data['text'].str.contains(
    r'\bшпиц|терьер|чихуахуа|овчарка|лабрадор|ретривер|хаски|бульдог|кокер|бигль|пудель|\
    ротвейлер|курцхаар\b'), 'CAT'] = 'Животные'

data.loc[data['text'].str.contains(
    r'\bперсидская кошка|сибирская кошка|невская кошка|абиссинская кошка|британская кошка|\
    шотландская кошка|шиншилла\b'), 'CAT'] = 'Животные'

data.loc[data['text'].str.contains(
    r'\bmüzik|vevo|songs|music|ozzy osbourn|acdc\b'), 'CAT'] = 'Музыка'

data.loc[data['text'].str.contains(
    r'\bмаша и медведь|фиксики|смешарики|лунтик|губка боб|spongebob|свинка пеппа|лего|lego\b'), 'CAT'] = 'Детское'
data.loc[data['text'].str.contains(
    r'\bkikido|super senya|mister max|vlad and niki|like nastya\b'), 'CAT'] = 'Детское'

data.loc[data['text'].str.contains(
    r'\bgaming|game|lazarbeam\b'), 'CAT'] = 'Игры'

data.loc[data['text'].str.contains(
    r'\bdota|dota2|дота|дота|gta|gta5|гта|гта5|standoff 2|standoff2|csgo|cs go\b'), 'CAT'] = 'Игры'
data.loc[data['text'].str.contains(
    r'\bигровой|battlefield|warcraft|blitz|rust\b'), 'CAT'] = 'Игры'
data.loc[data['text'].str.contains(
    r'\bworld of warships|skyrim|pubg|gameplay|standoff|стандофф|stalker|gamming\b'), 'CAT'] = 'Игры'
data.loc[data['text'].str.contains(
    r'\bmortal kombat|pubgmobile|fortnite|ms rainbow\b'), 'CAT'] = 'Игры'

data.loc[data['text'].str.contains(
    r'\bkidbehindacamera|lukeerwin23|useful tips\b'), 'CAT'] = 'Люди и блоги'

data.loc[data['text'].str.contains(
    r'\bsaksham magic|ash magic\b'), 'CAT'] = 'Развлечения'

data.loc[data['text'].str.contains(
    r'\bvalorant|grand theft auto|escape from tarkov|hearthstone|genshin impact|dead by daylight|\
    apex legends\b'), 'CAT'] = 'Игры'
data.loc[data['text'].str.contains(
    r'\bheroes of might and magic|heroes3|heroes 3|league of legends|path of exile|асмр|\
    propnight|phasmophobia\b'), 'CAT'] = 'Игры'
data.loc[data['text'].str.contains(
    r'\bhalo infinite|lineage 2|сталкрафт|stalcraft|subnautica|diablo|teamfight tactics|hitman|fifa\b'), 'CAT'] = 'Игры'
data.loc[data['text'].str.contains(
    r'\bforza horizon|the last of us|black desert|teardown|dying light|tom clancy|crusader kings|\
    hearts of iron\b'), 'CAT'] = 'Игры'
data.loc[data['text'].str.contains(
    r'\bneed for speed|farming simulator|euro truck simulator|dayz|the sims|call of duty|football manager\b'), 'CAT'] = 'Игры'

data.loc[data['text'].str.contains(
    r'\bworld of tanks|amway921|wot|war thunder|afl|offshore fishing\b'), 'CAT'] = 'Мужские Хобби'
data.loc[data['text'].str.contains(
    r'\bdota 2|csgo|fortnite|league of legends|starcraft 2|pubg|overwatch|hearthstone|arena of valor|\
    heroes of the storm|rainbow six siege|counterstrike 16 |pubg mobile|rocket league|smite|starcraft brood war|\
    halo 5|call of duty black ops|call of duty modern warfare|crossfire\b'), 'CAT'] = 'Киберспорт'
data.loc[data['text'].str.contains(
    r'\Bdota 2|csgo|fortnite|league of legends|starcraft 2|pubg|overwatch|hearthstone|arena of valor|\
    heroes of the storm|rainbow six siege|counterstrike 16 |pubg mobile|rocket league|smite|\
    starcraft brood war|halo 5|call of duty black ops|call of duty modern warfare|crossfire\b'), 'CAT'] = 'Киберспорт'

data.loc[data['text'].str.contains(
    r'\bdota 2|csgo|fortnite|league of legends|starcraft 2|pubg|overwatch|hearthstone|arena of valor|\
    heroes of the storm|rainbow six siege|counterstrike 16 |pubg mobile|rocket league|smite|\
    starcraft brood war|halo 5|call of duty black ops|call of duty modern warfare|crossfire\B'), 'CAT'] = 'Киберспорт'

data.loc[data['text'].str.contains(
    r'\ba4|влад бумага|майнкрафт|minecraft|а4|а4|minecraft|майнкрафт|himan|mamix|kid|kids\b'), 'CAT'] = 'Детское'
data.loc[data['text'].str.contains(
    r'\bнаруто|боруто|toys|toy|игрушки|егэ|гдз|bros show|lady kapa|anime|smg3|smg4|the mik maks\b'), 'CAT'] = 'Детское'
data.loc[data['text'].str.contains(
    r'\bminecr|майнкр|математи\B'), 'CAT'] = 'Детское'
data.loc[data['text'].str.contains(
    r'\bнаруто|майнкрафт|minecraft\B'), 'CAT'] = 'Детское'
data.loc[data['text'].str.contains(r'\Bminecraft\B'), 'CAT'] = 'Детское'
data.loc[data['text'].str.contains(r'\bminecr\B'), 'CAT'] = 'Детское'

data.loc[data['text'].str.contains(
    r'\bnursery rhymes|вильям бруно|кейт eva tv|chad wild clay|daniel gizmo|123 go\b'), 'CAT'] = 'Детское'
data.loc[data['text'].str.contains(
    r'\bpolitical|congress|trump|clinton|obama|congressional house|senate|legislation roll\b'), 'CAT'] = 'Новости и политика'

In [74]:
data.drop_duplicates(subset=['URL'], inplace = True)

In [75]:
data.head()

,URL,Сontent,Keywords,Name,text,language_code,language,CAT
0,http://youtube.com/channel/UCIPi6u4H3NZFuffKAl...,Assalomu alaykumKanalimizga xush kelibsizАЗИЗ ...,"трудовые мигранты"" ""аср медиа"" ""asr media"" ""по...",ASR MEDIA,assalomu alaykumkanalimizga xush kelibsizазиз ...,ru,Russian,Новости и политика
1,http://youtube.com/channel/UCjdHWLIKJNHsqaNJAF...,Официальный сайт : https://sho...,"qiziqarli videolar"" haqida ""shov-shuv uz",Shov- Shuv UZ,официальный сайт httpsshovshu...,ru,Russian,Развлечения
2,http://youtube.com/channel/UCSU6G0iZ5MI7bIM8pT...,Реклама на канале: pr@rumonkey.ruСмотрите шоу ...,"дорого богато"" обзор ""самый дорогой"" приветтач...",ПриветТачка,реклама на канале prrumonkeyruсмотрите шоу про...,ru,Russian,Кино и Сериалы
3,http://youtube.com/channel/UC2ADOzFv97QSk5NTN_...,NaN,сериалы фильмы кино новинки комедии мелодрамы ...,КиноПробка,nan сериалы фильмы кино новинки комедии мелодр...,ru,Russian,Спорт
5,http://youtube.com/channel/UCrGdxtTRGxAdMtz9xL...,«Прокурорская проверка» — это детективная реал...,NaN,Прокурорская Проверка,прокурорская проверка это детективная реалити...,ru,Russian,Кино и Сериалы


In [80]:
# Pivot CAT channel
pvivot_cat = data.pivot_table(
    index=['CAT'],
    values=['URL'],
    aggfunc='count').reset_index()

pvivot_cat.sort_values(['URL'], ascending=[False]).head(15)

,CAT,URL
11,Музыка,10807
10,Мужские Хобби,8257
12,Новости и политика,6097
4,Игры,4662
0,Детское,4386
7,Кино и Сериалы,4232
15,Развлечения,3549
9,Люди и блоги,3122
17,Спорт,2255
2,Животные,2187


In [79]:
# Pivot language channel
pvivot_language = data.pivot_table(
    index=['language'],
    values=['URL'],
    aggfunc='count').reset_index()

pvivot_language.sort_values(['URL'], ascending=[False]).head(15)

,language,URL
75,Russian,34185
22,English,17984
89,Ukrainian,1877
80,Spanish,1711
87,Turkish,1582
43,Kazakh,1480
3,Arabic,1220
76,Serbian,1210
13,Bulgarian,902
30,German,865


## Save file

In [76]:
data.to_pickle("channel_category.pkl")  